In [ ]:
import torch
import torch.nn as nn

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, \
    average_precision_score, classification_report, roc_curve, auc, top_k_accuracy_score, ndcg_score

In [ ]:
from tqdm import tqdm

In [ ]:
from torch.utils.data import TensorDataset,Dataset,DataLoader

In [ ]:
def my_metric(all_predictions, all_targets, all_probs):
    fpr, tpr, _ = roc_curve(all_targets, all_probs)
    auc_score = round(auc(fpr, tpr) * 100, 2)
    acc = round(accuracy_score(all_targets, all_predictions) * 100, 2)
    precision = round(precision_score(all_targets, all_predictions) * 100, 2)
    f1 = round(f1_score(all_targets, all_predictions) * 100, 2)
    recall = round(recall_score(all_targets, all_predictions) * 100, 2)
    matrix =  confusion_matrix(all_targets, all_predictions)
    target_names = ['non-vul', 'vul']
    report = classification_report(all_targets, all_predictions, target_names=target_names)
    result = f'auc: {auc_score} acc: {acc} precision: {precision} recall: {recall} f1: {f1} \nreport:\n{report}\nmatrix:\n{matrix}'
    print(result)  

In [ ]:
ep = 9

In [ ]:
train_X = torch.load(f'smote_data/stage_1_output/train_X_ep_{ep}.dt')
train_y = torch.load(f'smote_data/stage_1_output/train_y_ep_{ep}.dt')

In [ ]:
test_X = torch.load(f'smote_data/stage_1_output/test_X_ep_{ep}.dt')
test_y = torch.load(f'smote_data/stage_1_output/test_y_ep_{ep}.dt')

In [ ]:
print(train_X.shape,train_y.shape,test_X.shape,test_y.shape)

In [ ]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
train_X_sampled, train_y_sampled = sm.fit_resample(train_X, train_y)

In [ ]:
print(train_X_sampled.shape,train_y_sampled.shape)

In [ ]:
from sklearn.neural_network import MLPClassifier 
mlp = MLPClassifier(solver='adam',alpha=1e-5,hidden_layer_sizes=100,random_state=42,max_iter=1000)

In [ ]:
mlp.fit(train_X_sampled,train_y_sampled)

In [ ]:
predictions_mlp = mlp.predict(test_X)
probs_mlp = mlp.predict_proba(test_X)
probs_mlp = probs_mlp[:,1]

In [ ]:
my_metric(predictions_mlp, test_y.squeeze(1), probs_mlp)